**ReadMe**

The question / is answered in part 1.

#1. SoftMax

We select the policy via softmax.

In one sense, we can think of it as that if two actions yield a close enough optimal values. We think that we should give about the same probability of choosing both of them instead of giving all 100% into the slightly higher valued action.

This will help the problem of overconfidence in misspeciation adn the lack of competition between Q function approximation.

#1.1 Noise Addition


This same technique has also been used in various field of categorical prediction.

For example, in customer demand estimation, we usually assign utiity value for each item choice provided to a customer. If two items have about the same utility, we expect the probability of the item being chosen to be about the same. The explanation for this case is that there exists some intrinsic randomness and the utiltity approximation model just approximate the mean of the actual utility. We can see that in that sense, randomness does make sense. Otherwise, given the same set of offers, a customer will always choose the same item., which is not likely.

The idea of randomness may also stemmed from the decision theory in signal processing.

In signal processing, we assume that there may be gaussian noise addition during the sending process. This suggests that the fact that the observed signal is higher may not always come from the fact tat the original signal at the source is hugher, but may come from the noise being added.

However, due to the difficulty in dealing with gaussian cdf, in most modern machinelearning will use sfot max instead (which will be equivalent to other types of symmetric noise addition).

##1.2 Surrogate Risk

In our case on optimal policy in a more formal setting the justification on the randomness may be less obvious. However, we will see that it allows us to perform gradient descent easily.

At the assymptotic sense, we can see that soft max loss (cross entropy loss with softmax) will also be a surrogate risk for the mispeciation, meaning that training by minimizing the softmax loss will minimize the 0-1 loss.

However, this relies on the fact that each datapoint will have infinity occurrence and that the function approximators have great expressivity as in Tabular case, which can be not practical.

In this sense, the fact that a loss is a surrogate risk for another loss may be insufficient for getting a good performance.

#1.3 Score Function

\begin{align*}
  \pi(s,a; \theta) &= \frac{e^{\phi(s,a)^T \theta}}{ \sum_{b \in A(s)} e^{\phi(s,b)^T \theta}}\\
  &= \frac{e^{\sum_{i=1}^{m } \phi(s,a)_i \theta_i}}{ \sum_{b \in A(s)}  e^{\sum_{i=1}^{m } \phi(s,b)_i \theta_i}}
\end{align*}

\begin{align*}
  \log \pi(s,a; \theta) 
  &= \sum_{i=1}^{m } \phi(s,a)_i \theta_i - \log \left( \sum_{b \in A(s)}  e^{\sum_{i=1}^{m } \phi(s,b)_i \theta_i}\right)
\end{align*}

\begin{align*}
  \frac{\partial \log \pi(s,a; \theta)}{\partial \theta_i} &= \frac{\partial \phi(s,a)_i \theta_i}{\partial\theta_i} - \frac{1}{\sum_{b \in A(s)}  e^{\sum_{i=1}^{m } \phi(s,b)_i \theta_i}}
  \frac{\partial \sum_{b \in A(s)}  e^{\sum_{i=1}^{m } \phi(s,b)_i \theta_i}}{\partial\theta_i}\\
  &= \phi(s,a)_i - \frac{1}{\sum_{b \in A(s)}  e^{\sum_{i=1}^{m } \phi(s,b)_i \theta_i}} \sum_{b \in A(s)} \frac{\partial e^{\sum_{i=1}^{m } \phi(s,b)_i \theta_i}}{\partial\theta_i}\\
  &= \phi(s,a)_i - \frac{1}{\sum_{b \in A(s)}  e^{\sum_{i=1}^{m } \phi(s,b)_i \theta_i}} \sum_{b \in A(s)} \left(e^{\sum_{i=1}^{m } \phi(s,b)_i \theta_i} \frac{\partial {\sum_{i=1}^{m } \phi(s,b)_i \theta_i}}{\partial\theta_i}\right)\\
  &= \phi(s,a)_i - \frac{1}{\sum_{b \in A(s)}  e^{\sum_{i=1}^{m } \phi(s,b)_i \theta_i}} \sum_{b \in A(s)} \left(e^{\sum_{i=1}^{m } \phi(s,b)_i \theta_i} \phi(s,b)_i \right)\\
  &= \phi(s,a)_i -  \sum_{b \in A(s)} \left( \frac{e^{\sum_{i=1}^{m } \phi(s,b)_i \theta_i}}{\sum_{b \in A(s)}  e^{\sum_{i=1}^{m } \phi(s,b)_i \theta_i}} \phi(s,b)_i \right)\\
  &= \phi(s,a)_i -  \sum_{b \in A(s)} \left( \pi(s,b;\theta)\phi(s,b)_i \right)\\
\end{align*}

Therefore,
\begin{align*}
  \nabla_{\theta} \log \pi(s,a; \theta)
  &= \phi(s,a)-  \sum_{b \in A(s)} \left( \pi(s,b;\theta)\phi(s,b)\right)\\
\end{align*}

##1.4 Policy Gradient

Let's use linear policy.

\begin{align*}
  Q(s,a;w) = (\nabla_{\theta} \log \pi(s,a; \theta))^Tw
\end{align*}

Therefore,
\begin{align*}
  \mathbf{E}_{\pi}[Q(s,a;w)] &= \sum_{a\in A(s)} \pi(s,a;\theta) Q(s,a;w)\\
  &= \sum_{a\in A(s)}  \pi(s,a;\theta) (\nabla_{\theta} \log \pi(s,a; \theta))^Tw
\end{align*}

Consider the $i^\text{th}$ element.
\begin{align*}
  \mathbf{E}_{\pi}[Q_i(s,a;w)] &= \sum_{a\in A(s)}  \pi(s,a;\theta) \frac{\partial \log \pi(s,a; \theta)}{\partial \theta_i} w_i\\
  &= w_i \sum_{a\in A(s)} \left(\pi(s,a;\theta) \phi(s,a)_i -  \sum_{b \in A(s)} \left( \pi(s,a;\theta)\pi(s,b;\theta)\phi(s,b)_i \right) \right)\\
  &= w_i \left( \sum_{a\in A(s)} \pi(s,a;\theta) \phi(s,a)_i -  \sum_{b \in A(s)} \sum_{a\in A(s)}  \pi(s,a;\theta)\pi(s,b;\theta)\phi(s,b)_i \right)\\
  &= w_i \left( \sum_{a\in A(s)} \pi(s,a;\theta) \phi(s,a)_i -  \sum_{b \in A(s)}\pi(s,b;\theta)\phi(s,b)_i \right)\\ &= 0
\end{align*}